In [1]:
import logging, os
logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf

from pathlib import Path

import keras_tuner as kt

from importlib import reload
import hyper_model_creator
import keras_model_creator
import sentinel_utils

In [11]:
trials_dir = Path('trials', 'hyperband_resnet')
trial_metric = 'val_weightedf2score'

In [3]:
callbacks = [
    tf.keras.callbacks.TensorBoard(trials_dir.joinpath('board')),
    tf.keras.callbacks.EarlyStopping(
        monitor=trial_metric, patience=5, mode='max'
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor=trial_metric, factor=0.5, patience=8, min_lr=1e-6,
    ),
]

In [18]:
reload(hyper_model_creator)

hypermodel = hyper_model_creator.BuildHyperModel()

tuner = kt.Hyperband(
    hypermodel,
    objective=kt.Objective(trial_metric, direction='max'),
    directory=trials_dir.parent,
    project_name=trials_dir.name,
    hyperband_iterations=3,
    max_epochs=30,
    overwrite=False,
)
    
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]

# model = tuner.hypermodel.build(best_hp)

# tuner.search(
#     verbose=1, 
#     callbacks=callbacks
# )

Reloading Tuner from trials/hyperband_resnet/tuner0.json


In [19]:
followup_trials_dir = Path('trials', 'hyperband_resnet_followup')
trial_metric = 'val_weightedf2score'

followup_hypermodel = hyper_model_creator.BuildHyperModel(best_hp)

followup_tuner = kt.Hyperband(
    followup_hypermodel,
    objective=kt.Objective(trial_metric, direction='max'),
    directory=followup_trials_dir.parent,
    project_name=followup_trials_dir.name,
    hyperband_iterations=3,
    max_epochs=30,
    overwrite=False,
)

for p in best_hp.space:
    config = p.get_config()
    print(config)

# model = tuner.hypermodel.build(best_hp)

# model.fit(best_hp, model, epochs=50)


# model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

{'name': 'class_weight', 'default': True, 'conditions': []}
{'name': 'training_years', 'default': '2017_2018_2019', 'conditions': [], 'values': ['2017_2018_2019', '2017'], 'ordered': False}
{'name': 'kernel_regularizer', 'default': 'l1l2', 'conditions': [], 'values': ['l1l2', 'l1', 'l2'], 'ordered': False}
{'name': 'spatial_dropout', 'default': 0.3, 'conditions': [], 'values': [0.3, 0.1, 0.5], 'ordered': True}
{'name': 'activation', 'default': 'leaky_relu', 'conditions': [], 'values': ['leaky_relu', 'relu'], 'ordered': False}
{'name': 'pool_size', 'default': 4, 'conditions': [], 'values': [4, 2], 'ordered': True}
{'name': 'dropout', 'default': 0.3, 'conditions': [], 'values': [0.3, 0.1, 0.5], 'ordered': True}
{'name': 'bias_initializer', 'default': True, 'conditions': []}
{'name': 'learning_rate', 'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001, 0.01], 'ordered': True}
{'name': 'loss_function', 'default': 'binary_focal_crossentropy', 'conditions': [], 'values': ['binary_fo

In [ ]:
# tuner.results_summary(num_trials=2)

In [ ]:
# %load_ext tensorboard
# tb_path = str(trials_dir.joinpath('board'))
# %tensorboard --logdir "$tb_path"